In [1]:
# Here we replicate the resuls of Andrew L. Ferguson's Bayesian Approach to Free Energy Estimation
# Uncertainty quantification, and how these relate to the traditional Weighted Histogram Analysis Method
# (WHAM) used to calculate unbiased free energy surfaces from biased protein simulation data (a proecure
# know as Umbrella Sampling)

In [ ]:
# Step 1: Run Ferguson's scripts without any modification - generate the plots for alanine dipeptide
